## Task-list for the Omero
1. access to the dataset by OmeroID
2. extract chanell from multichanell file
3. choose particular Z from Z-stack
4. extract frames from T1 to T2
  

### Import Packages

In [1]:
from IPython import get_ipython
from omero.gateway import BlitzGateway
from omero.model import PlaneInfo
import matplotlib.pyplot as plt
from getpass import getpass
import numpy as np

Matplotlib is building the font cache; this may take a moment.


### OMERO Credentials

In [2]:
HOST = 'ome.evolbio.mpg.de'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()

Username:  pichugina
OMERO Password:  ········


True

### Connect to the data set

In [ ]:
data_set_id=2222
dataset = conn.getObject("Dataset", data_set_id)

print(dataset.getName(), dataset.getDescription())
print("#=====================================================#")

## helper function ##
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print("""%s%s:%s   \n Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName()))
    print("Z={},Chanells={} Frames={}".format(obj.getSizeZ(),obj.getSizeC(),obj.getSizeT()))
    
    size_x_obj = obj.getPixelSizeX(units=True)
    print(" Pixel Size X:", size_x_obj.getValue(), "(%s)" % size_x_obj.getSymbol())
    
    
    theC=obj.getSizeC()
    planeInfo = obj.getPrimaryPixels().copyPlaneInfo(theC=theC, theZ=0, theT=1)
    
    for pi in planeInfo:
        deltaT = pi.getDeltaT(units="SECOND")
        exposure = pi.getExposureTime(units="SECOND")
        if deltaT is not None:
            print('deltaT secs', deltaT.getValue())
        if exposure is not None:
            print('exposure secs', exposure.getValue())
    print("#=====================================================#")

def show_plane(obj,frame):
    pixels = obj.getPrimaryPixels()
    plane = pixels.getPlane(0, 0, frame) 
    fig,ax=plt.subplots(figsize=(20,20))
    ax.set_title(obj.getName())
    ax.imshow(plane,vmin=np.quantile(plane,0.02),vmax=np.quantile(plane,0.98),cmap='gray')
    
for image in dataset.listChildren():
    print_obj(image, 4)
    show_plane(image,20)
    

### OMERO Image ID

In [ ]:
# To be modified
# ex: Select an Image from the dataset named 'PTRE' and enter its Id
image_id =2405691
image = conn.getObject("Image", image_id)

print(image.getName(), image.getDescription())
print("Z={},Chanells={} Frames={}".format(image.getSizeZ(),image.getSizeC(),image.getSizeT()))

print("Chanel")
for i, ch in enumerate(image.getChannels()):
    print(ch.getLabel())
    # Return as output. Key is string, value is rtype

In [ ]:
#plane = pixels.getPlane(z, c, t) 
pixels = image.getPrimaryPixels()
plane = pixels.getPlane(0, 0, 20) 

fig,ax=plt.subplots(figsize=(20,20))
ax.imshow(plane,vmin=np.quantile(plane,0.02),vmax=np.quantile(plane,0.98))

### Retrieve imagestack

In [ ]:
# Get a T-stack of tiles. Using getTiles or getPlanes (see below) returns
# a generator of data (not all the data in hand) The RawPixelsStore is
# only opened once (not closed after each plane) Alternative is to use
# getPlane() or getTile() multiple times - slightly slower.
c, z = 0, 0                 # First channel and timepoint
size_t=image.getSizeT()
plane = pixels.getPlane(0, 0, 0) 
tile = (0, 0, plane.shape[1], plane.shape[0])     # x, y, width, height of tile

# list of [ (0,0,0,(x,y,w,h)), (1,0,0,(x,y,w,h)), (2,0,0,(x,y,w,h))... ]
zct_list = [(z, c, t, tile) for t in range(size_t)]
print("\nZ stack of tiles:")
planes = pixels.getTiles(zct_list)
for i, p in enumerate(planes):
    print("Tile:", zct_list[i], " min:", p.min(),\
        " max:", p.max(), " sum:", p.sum())

### Close the connection to the OMERO server

In [ ]:
conn.close()

### License
Copyright (C) 2019 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.